# Transformation Digitale

Le but de ce projet est d'accompagner le marchand de vins de renom dans sa ***transformation digitale*** afin que ces ventes sur Internet soient prises en compte et surtout de valoriser ce marché du numérique qui offre plusieurs opportunités.

Mon rôle étant que Data Scientist est de l'aider à bâtir des outils nécessaires pour l'analyse de ces ventes sur Internet qui, jusques là étaient impossibles d'analyser pour manque de rapprochement entre les exports des tables de ventes sur Internet et ceux fait hors ligne.

![caviste](https://www.espacemarconi.fr/wp-content/uploads/2019/04/espace-marconi-459-1080x675.jpg)

# Table de Matières

# 1. Définition du problème

- L'objectif ici est de prédire le `prix` de vin en fonction des variables `quantitives`. Ensuite, construire les `profils de vins` autour de et de décrire les cluster obenus

- Nous avons à notre portée plusieurs jeux de données, notamment sur les réferences des produits en boutique et celles en lignes.

- Nous commenceront de rien pour construire au fur et à mesure une solution satisfaisante ayant de bonnes performances.

- Il s'agit d'unn problème de `Machine Learning` et plus précisement d'une `régression` pour le premier cas qui est la prédiction du prix de vins et d'un deuxième cas d'une `classification` pour la construction des profils de vins.

- Nous évaluerons le moddèle du premier cas en utilisant le métrique `R2` et pour le sécond cas le `Score F1` pour ensuire voir d'autres métriques comme `l'accuracy`.

# 2. Importation des librairies et acquisition des données

## 2.1. Importation des librairies

In [1]:
# Manipulation des données
import numpy as np
import pandas as pd

# Visualisation des données
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Package de modélisation
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

# Package d'évaluation de modèle
from sklearn.metrics import r2_score, f1_score

# Package pour intertpréter les modèles
import shap

## 2.2 Acquisition de données

In [2]:
# Charger les données dans le notebook
wine_df = pd.read_csv("wine_data.csv")

In [3]:
# Afficher la taille des données
print(f"La base fait {wine_df.shape[0]} lignes et {wine_df.shape[1]} colonnes")
# Afficher les 5 premières lignes des données
wine_df.head()

La base fait 896 lignes et 5 colonnes


,total_sales,onsale_web,price,stock_quantity,stock_status
0,10.0,1,25.0,0,instock
1,6.0,1,24.2,0,outofstock
2,0.0,1,34.3,0,outofstock
3,0.0,1,20.8,0,outofstock
4,3.0,1,14.1,0,outofstock


# 3. Feature Engineering

- **Calcul du chiffre d'affaire réalisé par produit**

In [4]:
# Calculer le chiffre d'affaire par produit
wine_df["revenue"] = wine_df["price"] * wine_df["total_sales"]

In [5]:
# Vérification des ajouts
wine_df.head()

,total_sales,onsale_web,price,stock_quantity,stock_status,revenue
0,10.0,1,25.0,0,instock,250.0
1,6.0,1,24.2,0,outofstock,145.2
2,0.0,1,34.3,0,outofstock,0.0
3,0.0,1,20.8,0,outofstock,0.0
4,3.0,1,14.1,0,outofstock,42.3


- **Calcul du chiffre d'affaire total réalisé**

In [6]:
# Calculer le chiffre total
total_revenue = sum(wine_df["revenue"])

In [7]:
# Afficher le chiffre d'affaire total réalisé
print(f"Vous avez réalisés {total_revenue} € de chiffre d'affaires 😊")

Vous avez réalisés 70568.6 € de chiffre d'affaires 😊


# 4. Régression Linéaire

In [8]:
# Définir le SEED
SEED = 42
# Diviser les données en jeu de train et de test
df_train, df_test = train_test_split(wine_df, 
                                     test_size=0.2, 
                                     random_state=SEED)

In [9]:
# Vérifier les taille des deux jeux de données
print(f"Le jeu de train fait : {df_train.shape}\n")
print(f"Le jeu de test fait : {df_test.shape}")

Le jeu de train fait : (716, 6)

Le jeu de test fait : (180, 6)


- **Création des jeux de train et de test en matrices et vecteurs**

In [10]:
# Créer la matrice et le vecteur de train
X_train = df_train.drop(columns=["price"])
y_train = df_train["price"]

# Créer la matrice et le vecteur de test
X_test = df_test.drop(columns=["price"])
y_test = df_test["price"]

- **Création des lites de variables par selon leurs catégories [numérique ou catégorielle]**

In [11]:
# La liste des variables numériques
num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
# Liste des variables catégorielles
cat_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

## 4.1 Pipeline d'entrainement de modèles

Afin de bien mener cette régression linéaire, j'ai fait le choix d'utiliser des pipelines pour faire les traitements suivants :

- *`Encodage` de variables catégorielles en utilisant `OneHotEncoder`*;
- *`Normaliser` les données en avec `RobustScaler`*

In [12]:
# Définir la fonction d'entrainement de modèles
def fit_model(estimator, X_train, y_train):
    """Cette fonction permet d'entrainer un modèle de Machine Learning en fonction
       de l'estimateur, les données d'entrainement  et renvoi un modèle entrainé

    Args:
        estimator (Classifieur): C'est l'estimateur qu'il faut utiliser 
        X (DataFrame): Les données d'explicatives d'entrainement
        y (Vecteur): Le vecteur explicatif d'entrainement

    Returns:
        Classifieur: Le modèle entrainé
    """
    
    # Séparer les données numériques et catégorielles
    num_cols = X_train.select_dtypes(include=np.number).columns
    cat_cols = X_train.select_dtypes(exclude=np.number).columns
    
    # Traitement des variables numériques
    num_pipeline = make_pipeline(RobustScaler()
                                )
    #Traiter les variables catégorielles
    cat_pipeline = make_pipeline(OneHotEncoder()
                                )
    
    # Combinaison des deux pipelines
    preprocessor = make_column_transformer(
        (num_pipeline, num_cols), 
        (cat_pipeline, cat_cols)
    )
    
    # Construction de modèle
    model = Pipeline(steps=[
        ("preprocessor", preprocessor), 
        ("estimator", estimator)]
    )
    
    # Renvoyer le modèle construit et entraîné
    return model.fit(X_train, y_train)

In [13]:
# Définir la fonction d'évaluation de modèles

def model_evaluation(model, X_train, y_train, X_test, y_test):
    
    """Cette fonction permet d'évaluer la performance d'un modèle

    Args:
        model (Classifieur): L'estimateur à utiliser pour évaluer le modèle
        X_train (DataFrame): Les données explicative d'entrainement
        y_train (Vecteur): Le vecteur target d'entrainement
        X_test (DataFrame): Les données explicatives de test
        y_test (Vecteur): Le vecteur target de test

    Returns:
        float: Ce sont les scores f1 de train et de test qui sont renvoyés
    """
    
    # Faire des prédictions sur les deux jeux de données
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculer le socre R2 sur les deux jeux
    r2_score_train = r2_score(y_train, y_train_pred)
    r2_score_test = r2_score(y_test, y_test_pred)
    
    # Renvoyer les scores R2 calculés
    return r2_score_train, r2_score_test

In [14]:
# Intancier le modèle
reg = LinearRegression(n_jobs=2)

In [17]:
# Entrainer la régression linéaire
reg_model = fit_model(reg, X_train, y_train)

# Evaluer le modèle de régression linéaire
evaluation = model_evaluation(reg_model, X_train, y_train, X_test, y_test)

- **Affichage des performance d'entrainement et d'évaluation**

In [20]:
# Afficher les scores
print(f"Score R2 de {round(evaluation[0], 2)} au train et de {round(evaluation[1], 2)} au test")

Score R2 de 0.22 au train et de 0.05 au test


# 5. Construction des profils de vins